In [339]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [340]:
data=pd.read_csv("Churn_Modelling.csv")

In [341]:
data.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [342]:
data.tail(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1
9999,10000,15628319,Walker,792,France,Female,28,4,130142.79,1,1,0,38190.78,0


In [343]:
data.shape

(10000, 14)

In [344]:
data.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [345]:
# Preprocess the data
# drop the irrelavant columns

data=data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [346]:
data.head(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [347]:
cat_features=[feature for feature in data.columns if data[feature].dtypes=='object']
num_features=[feature for feature in data.columns if data[feature].dtypes!='object']

In [348]:
print(cat_features)
print(num_features)

['Geography', 'Gender']
['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited']


In [349]:
# Categorical and Numerical Features

# Encoding

label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])

In [350]:
data['Gender']

0       0
1       0
2       0
3       0
4       0
       ..
9995    1
9996    1
9997    0
9998    1
9999    0
Name: Gender, Length: 10000, dtype: int64

In [351]:
from sklearn.preprocessing import OneHotEncoder

In [352]:
onehotencoder_geo=OneHotEncoder()

In [353]:
geo_encoder=onehotencoder_geo.fit_transform(data[['Geography']])

In [354]:
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [355]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [356]:
onehotencoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [357]:
geo_encoder_df=pd.DataFrame(geo_encoder.toarray(), columns=onehotencoder_geo.get_feature_names_out(['Geography']))

In [358]:
data['Geography'].dtypes

dtype('O')

In [359]:
# Combine the one hot encoder columns with the original data

data=pd.concat([data.drop('Geography', axis=1), geo_encoder_df], axis=1)

In [360]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [361]:
data.dtypes

CreditScore            int64
Gender                 int64
Age                    int64
Tenure                 int64
Balance              float64
NumOfProducts          int64
HasCrCard              int64
IsActiveMember         int64
EstimatedSalary      float64
Exited                 int64
Geography_France     float64
Geography_Germany    float64
Geography_Spain      float64
dtype: object

In [362]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CreditScore        10000 non-null  int64  
 1   Gender             10000 non-null  int64  
 2   Age                10000 non-null  int64  
 3   Tenure             10000 non-null  int64  
 4   Balance            10000 non-null  float64
 5   NumOfProducts      10000 non-null  int64  
 6   HasCrCard          10000 non-null  int64  
 7   IsActiveMember     10000 non-null  int64  
 8   EstimatedSalary    10000 non-null  float64
 9   Exited             10000 non-null  int64  
 10  Geography_France   10000 non-null  float64
 11  Geography_Germany  10000 non-null  float64
 12  Geography_Spain    10000 non-null  float64
dtypes: float64(5), int64(8)
memory usage: 1015.8 KB


In [363]:
data.isnull().sum()

CreditScore          0
Gender               0
Age                  0
Tenure               0
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
Geography_France     0
Geography_Germany    0
Geography_Spain      0
dtype: int64

In [364]:
data.duplicated().sum()

np.int64(0)

In [365]:
data.describe()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,650.528800,0.545700,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700,0.501400,0.250900,0.247700
std,96.653299,0.497932,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769,0.500023,0.433553,0.431698
min,350.000000,0.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000,0.000000,0.000000,0.000000
25%,584.000000,0.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000,0.000000,0.000000,0.000000
50%,652.000000,1.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000,1.000000,0.000000,0.000000
75%,718.000000,1.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000,1.000000,1.000000,0.000000
max,850.000000,1.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000,1.000000,1.000000,1.000000


In [366]:
import pickle

In [367]:
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)
    
with open('label_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehotencoder_geo, file)

In [368]:
# Dividing the dataset into independent and dependent features

data.columns
X=data.drop('Exited', axis=1)
y=data['Exited']

In [369]:
X

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,0.0,1.0,0.0


In [370]:
y

0       1
1       0
2       1
3       0
4       0
       ..
9995    0
9996    0
9997    1
9998    1
9999    0
Name: Exited, Length: 10000, dtype: int64

In [371]:
# Splitting the data into training and testing

X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3, random_state=23)

In [372]:
X_train.shape, X_test.shape

((7000, 12), (3000, 12))

In [373]:
# Scaling the features

scaler=StandardScaler()

X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [374]:
X_train

array([[ 2.02129476,  0.91660692,  2.09753592, ..., -0.99771689,
        -0.57559072,  1.72155741],
       [ 0.20507906,  0.91660692, -0.85430682, ..., -0.99771689,
        -0.57559072,  1.72155741],
       [ 0.52498069,  0.91660692,  0.09790052, ...,  1.00228833,
        -0.57559072, -0.58086939],
       ...,
       [-0.25929428,  0.91660692, -1.61607268, ...,  1.00228833,
        -0.57559072, -0.58086939],
       [ 1.18542276,  0.91660692, -0.18776168, ..., -0.99771689,
         1.73734559, -0.58086939],
       [ 0.4011478 ,  0.91660692,  3.24018472, ..., -0.99771689,
        -0.57559072,  1.72155741]])

In [375]:
X_test

array([[-0.98165279,  0.91660692, -0.75908608, ...,  1.00228833,
        -0.57559072, -0.58086939],
       [ 1.17510335, -1.0909802 ,  0.00267978, ..., -0.99771689,
         1.73734559, -0.58086939],
       [-1.90008005, -1.0909802 ,  1.62143225, ..., -0.99771689,
         1.73734559, -0.58086939],
       ...,
       [-1.42538731, -1.0909802 ,  0.09790052, ..., -0.99771689,
         1.73734559, -0.58086939],
       [ 0.76232706,  0.91660692, -0.56864462, ..., -0.99771689,
         1.73734559, -0.58086939],
       [-0.20769724,  0.91660692, -1.23518975, ...,  1.00228833,
        -0.57559072, -0.58086939]])

In [376]:
with open('scaling.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [377]:
## ANN Implementation

import tensorflow as tf

In [378]:
import tensorflow as tf

In [379]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

import datetime

In [380]:
X_train.shape[1],

(12,)

In [381]:
# Build ANN Model

model=Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), # HL1 connected with i/p layer
    Dense(32, activation='relu'),  # HL2
    Dense(1, activation='sigmoid')  # O/p Layer
])

/Users/siddanthapusandeep/End-to-End-Deep-Learning-Project/.conda/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [382]:
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_21 (Dense)                │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [383]:
import tensorflow as tf

In [384]:
opt=tf.keras.optimizers.Adam(learning_rate=0.01)

In [385]:
# Compile the Model

model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [386]:
# setup the tensorboard

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir, histogram_freq=1)

In [387]:
# Setup Early Stopping

early_stopping=EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [388]:
# Train the Model

history=model.fit(
    X_train, y_train, validation_data=(X_test, y_test), epochs=100,
    callbacks=[tensorflow_callback, early_stopping]
)

Epoch 1/100
  1/219 ━━━━━━━━━━━━━━━━━━━━ 1:31 422ms/step - accuracy: 0.7188 - loss: 0.6098

219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8246 - loss: 0.4191 - val_accuracy: 0.8543 - val_loss: 0.3652
Epoch 2/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 842us/step - accuracy: 0.8524 - loss: 0.3604 - val_accuracy: 0.8603 - val_loss: 0.3479
Epoch 3/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 836us/step - accuracy: 0.8537 - loss: 0.3571 - val_accuracy: 0.8580 - val_loss: 0.3471
Epoch 4/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 836us/step - accuracy: 0.8553 - loss: 0.3417 - val_accuracy: 0.8607 - val_loss: 0.3520
Epoch 5/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 833us/step - accuracy: 0.8648 - loss: 0.3437 - val_accuracy: 0.8627 - val_loss: 0.3460
Epoch 6/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 834us/step - accuracy: 0.8696 - loss: 0.3224 - val_accuracy: 0.8603 - val_loss: 0.3520
Epoch 7/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 850us/step - accuracy: 0.8657 - loss: 0.3298 - val_accuracy: 0.8573 - val_loss: 0.3458
Epoch 8/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 829us/step - accuracy: 0.8625 - loss: 0.3414 - val_ac

In [389]:
model.save('model.h5')

In [390]:
# Load Tensorboard Extension

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [391]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 2304), started 0:40:12 ago. (Use '!kill 2304' to kill it.)

In [392]:
# Load the Pickle file

import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [393]:
# Load the trained model, scaler_pickle, onehot

model1=load_model('model.h5')

# load the encoder and scaler

with open('label_encoder_geo.pkl', mode='rb') as file:
    label_encoder_geo=pickle.load(file)
    
with open('label_encoder_gender.pkl', mode='rb') as file:
    label_encoder_gender=pickle.load(file)

with open('scaling.pkl', mode='rb') as file:
    scaler=pickle.load(file)


In [394]:
data.columns

Index(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited',
       'Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype='object')

In [395]:
# Example Input data

input_data={
    'CreditScore': 600,
    'Gender' : 'Male',
    'Geography': 'France',
    'Age' : 23,
    'Tenure': 3,
    'Balance': 600000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
    
}

In [396]:
geo_encoded=onehotencoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoder_df=pd.DataFrame(geo_encoded, columns=onehotencoder_geo.get_feature_names_out(['Geography']))

geo_encoder_df

/Users/siddanthapusandeep/End-to-End-Deep-Learning-Project/.conda/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [397]:
input_df=pd.DataFrame([input_data])

In [398]:
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])


input_df

,CreditScore,Gender,Geography,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,1,France,23,3,600000,2,1,1,50000


In [400]:
# Concatination one hot encoded

input_df=pd.concat([input_df.drop("Geography", axis=1), geo_encoder_df], axis=1)

input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,23,3,600000,2,1,1,50000,1.0,0.0,0.0


In [401]:
# Scaling the input data

input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.51727946,  0.91660692, -1.52085195, -0.7003816 ,  8.41198864,
         0.8096538 ,  0.6430943 ,  0.96794215, -0.86053365,  1.00228833,
        -0.57559072, -0.58086939]])

In [402]:
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


array([[0.35805023]], dtype=float32)

In [403]:
prediction_proba=prediction[0][0]
prediction_proba

np.float32(0.35805023)

In [404]:
if prediction_proba>0.5:
    print("The Customer is likely to churn")
else:
    print("The Customer is not likely to churn")

The Customer is not likely to churn
